## Building the Predictive Model of Pollution Air Quality Indices
#### Molly McNamara

### Import basic packages and dataset

In [1]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import Imputer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn import linear_model
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegressionCV
from sklearn.feature_selection import RFE


from IPython.display import display,HTML
pd.set_option("display.max.columns",500)
pd.set_option("display.max.rows",500)
%matplotlib inline
pollution = pd.read_csv('~/Desktop/weatherpollution.csv', index_col='Unnamed: 0')
pollution['Date_Local']= pd.to_datetime(pollution['Date_Local'],  errors='raise', format='%Y/%m/%d')
the7cities = ['New York', 'Los Angeles', 'Houston', 'Phoenix', 'Philadelphia', 'San Diego', 'Dallas']
pollution7 = pollution[pollution['City'].isin(the7cities)]
pollution7.describe()

/Users/mollymcnamara/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/mollymcnamara/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (28,29,30,31,35,36,39,40,41,42,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Site_Num,NO2_Mean,NO2_1stMaxValue,NO2_1stMaxHour,NO2_AQI,O3_Mean,O3_1stMaxValue,O3_1stMaxHour,O3_AQI,SO2_Mean,SO2_1stMaxValue,SO2_1stMaxHour,SO2_AQI,CO_Mean,CO_1stMaxValue,CO_1stMaxHour,CO_AQI,TempAvg,TempMax,TempMin,Elevation,Latitude,Longitude,AvgRelHumid,AvgDewPointTemp,AvgWetBulbTemp,Sunrise,Sunset,AvgStationPressure,AvgSeaLevelPressure,SustainedWindSpeed,SustainedWindDirection
count,56073.000000,56073.000000,56073.000000,56073.000000,56073.000000,56073.000000,56073.000000,56073.000000,56073.000000,56073.000000,56073.000000,56073.000000,56073.000000,56073.000000,56073.000000,56073.000000,56073.000000,55576.00000,55576.000000,55576.000000,48244.000000,48244.000000,48244.000000,43531.000000,43533.000000,43533.000000,48244.000000,48244.000000,48244.000000,43533.000000,48098.000000,48098.000000
mean,1705.768445,19.922606,36.264774,11.982469,34.208639,0.022574,0.036400,10.131311,32.910420,2.662308,5.484707,9.945638,8.588099,0.494266,0.862853,8.701354,8.332008,72.51049,73.152458,56.454027,76.495902,34.900695,-99.066550,64.116032,40.550020,49.291572,581.320185,1801.010426,29.652541,30.026183,18.205996,215.329951
std,2519.681700,10.817550,16.494427,7.769104,15.843286,0.011053,0.015641,3.991167,18.404656,3.407479,7.382645,6.685631,11.776887,0.379911,0.727501,6.736504,6.908859,14.16188,16.696224,14.699040,118.393967,3.774871,17.806223,19.700994,14.711799,11.803444,88.062569,90.918767,0.461684,0.170512,5.831168,90.989462
min,4.000000,-0.152174,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.725155,-1.300000,0.000000,0.000000,-0.400000,-0.400000,0.000000,0.000000,0.00000,13.000000,2.000000,3.000000,29.980000,-118.388800,5.000000,-11.000000,6.000000,423.000000,1628.000000,28.240000,29.170000,5.000000,10.000000
25%,124.000000,11.826087,24.300000,6.000000,23.000000,0.014083,0.025000,9.000000,22.000000,0.531677,1.300000,5.000000,1.000000,0.239584,0.389500,3.500000,3.000000,66.00000,63.000000,47.000000,3.400000,32.733600,-117.183100,51.000000,30.000000,42.000000,521.000000,1728.000000,29.510000,29.930000,14.000000,150.000000
50%,1010.000000,18.375000,36.000000,10.000000,34.000000,0.021958,0.036000,10.000000,31.000000,1.652083,3.000000,8.000000,4.000000,0.393750,0.650000,7.500000,7.000000,73.00000,73.000000,57.000000,29.000000,33.427700,-96.855500,67.000000,37.000000,46.000000,559.000000,1815.000000,29.790000,30.060000,17.000000,250.000000
75%,2007.000000,26.487500,46.000000,20.000000,43.000000,0.030250,0.046000,11.000000,40.000000,3.467702,6.650000,14.000000,10.000000,0.637500,1.100000,11.500000,10.000000,84.00000,84.000000,67.000000,29.600000,39.873270,-75.226780,83.000000,51.000000,58.000000,646.000000,1859.000000,29.960000,30.100000,21.000000,280.000000
max,9997.000000,98.130435,163.000000,23.000000,113.000000,0.079458,0.130000,23.000000,206.000000,53.481250,174.500000,23.000000,176.000000,4.111795,7.850000,23.000000,66.000000,106.00000,119.000000,96.000000,337.400000,40.779200,-73.880000,99.000000,78.000000,80.000000,733.000000,1942.000000,30.730000,30.860000,62.000000,360.000000


### Prepare the data (outcome variable --> category)

#### Create AQI categories

In [2]:
def AQI(x):
    if 0 <= x <= 50:
        return 'Good'
    elif 50 < x <= 100:
        return 'Moderate'
    elif 100 < x <= 150:
        return 'Unhealthy for Sensitive Groups'
    elif 150 < x <= 200:
        return 'Unhealthy'
    elif 200 < x <= 300:
        return 'Very Unhealthy'
    elif 300 < x <= 500:
        return 'Hazardous'
    

pollution7['OzoneAQI'] = pollution7['O3_AQI'].apply(AQI)
pollution7['NitrogenDioxideAQI'] = pollution7['NO2_AQI'].apply(AQI)
pollution7['CarbonMonoxideAQI'] = pollution7['CO_AQI'].apply(AQI)
pollution7['SulfurDioxideAQI'] = pollution7['SO2_AQI'].apply(AQI)

/Users/mollymcnamara/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/mollymcnamara/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/mollymcnamara/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

#### Convert them to category type and code numerically

In [3]:
ordered_AQI = ['Good', 'Moderate', 'Unhealthy for Sensitive Groups', 'Unhealthy', 'Very Unhealthy', 'Hazardous']
pollution7['OzoneAQI'] = pollution7['OzoneAQI'].astype('category', ordered=True,
  categories=ordered_AQI).cat.codes
pollution7['NitrogenDioxideAQI'] = pollution7['NitrogenDioxideAQI'].astype('category', ordered=True,
  categories=ordered_AQI).cat.codes
pollution7['CarbonMonoxideAQI'] = pollution7['CarbonMonoxideAQI'].astype('category', ordered=True,
  categories=ordered_AQI).cat.codes
pollution7['SulfurDioxideAQI'] = pollution7['SulfurDioxideAQI'].astype('category', ordered=True,
  categories=ordered_AQI).cat.codes
pollution7.dtypes

/Users/mollymcnamara/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/mollymcnamara/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/mollymcnamara/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  """
/Users/mollymcnamara/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: Set

Site_Num                           int64
Date_Local                datetime64[ns]
State                             object
County                            object
City                              object
NO2_Mean                         float64
NO2_1stMaxValue                  float64
NO2_1stMaxHour                   float64
NO2_AQI                          float64
O3_Mean                          float64
O3_1stMaxValue                   float64
O3_1stMaxHour                    float64
O3_AQI                           float64
SO2_Mean                         float64
SO2_1stMaxValue                  float64
SO2_1stMaxHour                   float64
SO2_AQI                          float64
CO_Mean                          float64
CO_1stMaxValue                   float64
CO_1stMaxHour                    float64
CO_AQI                           float64
TempAvg                          float64
TempMax                          float64
TempMin                          float64
Elevation       

In [4]:
pollution7.describe()

,Site_Num,NO2_Mean,NO2_1stMaxValue,NO2_1stMaxHour,NO2_AQI,O3_Mean,O3_1stMaxValue,O3_1stMaxHour,O3_AQI,SO2_Mean,SO2_1stMaxValue,SO2_1stMaxHour,SO2_AQI,CO_Mean,CO_1stMaxValue,CO_1stMaxHour,CO_AQI,TempAvg,TempMax,TempMin,Elevation,Latitude,Longitude,AvgRelHumid,AvgDewPointTemp,AvgWetBulbTemp,Sunrise,Sunset,AvgStationPressure,AvgSeaLevelPressure,SustainedWindSpeed,SustainedWindDirection,OzoneAQI,NitrogenDioxideAQI,CarbonMonoxideAQI,SulfurDioxideAQI
count,56073.000000,56073.000000,56073.000000,56073.000000,56073.000000,56073.000000,56073.000000,56073.000000,56073.000000,56073.000000,56073.000000,56073.000000,56073.000000,56073.000000,56073.000000,56073.000000,56073.000000,55576.00000,55576.000000,55576.000000,48244.000000,48244.000000,48244.000000,43531.000000,43533.000000,43533.000000,48244.000000,48244.000000,48244.000000,43533.000000,48098.000000,48098.000000,56073.000000,56073.000000,56073.000000,56073.000000
mean,1705.768445,19.922606,36.264774,11.982469,34.208639,0.022574,0.036400,10.131311,32.910420,2.662308,5.484707,9.945638,8.588099,0.494266,0.862853,8.701354,8.332008,72.51049,73.152458,56.454027,76.495902,34.900695,-99.066550,64.116032,40.550020,49.291572,581.320185,1801.010426,29.652541,30.026183,18.205996,215.329951,0.099228,0.132435,0.001213,0.016889
std,2519.681700,10.817550,16.494427,7.769104,15.843286,0.011053,0.015641,3.991167,18.404656,3.407479,7.382645,6.685631,11.776887,0.379911,0.727501,6.736504,6.908859,14.16188,16.696224,14.699040,118.393967,3.774871,17.806223,19.700994,14.711799,11.803444,88.062569,90.918767,0.461684,0.170512,5.831168,90.989462,0.347624,0.349738,0.034803,0.137558
min,4.000000,-0.152174,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.725155,-1.300000,0.000000,0.000000,-0.400000,-0.400000,0.000000,0.000000,0.00000,13.000000,2.000000,3.000000,29.980000,-118.388800,5.000000,-11.000000,6.000000,423.000000,1628.000000,28.240000,29.170000,5.000000,10.000000,0.000000,0.000000,0.000000,0.000000
25%,124.000000,11.826087,24.300000,6.000000,23.000000,0.014083,0.025000,9.000000,22.000000,0.531677,1.300000,5.000000,1.000000,0.239584,0.389500,3.500000,3.000000,66.00000,63.000000,47.000000,3.400000,32.733600,-117.183100,51.000000,30.000000,42.000000,521.000000,1728.000000,29.510000,29.930000,14.000000,150.000000,0.000000,0.000000,0.000000,0.000000
50%,1010.000000,18.375000,36.000000,10.000000,34.000000,0.021958,0.036000,10.000000,31.000000,1.652083,3.000000,8.000000,4.000000,0.393750,0.650000,7.500000,7.000000,73.00000,73.000000,57.000000,29.000000,33.427700,-96.855500,67.000000,37.000000,46.000000,559.000000,1815.000000,29.790000,30.060000,17.000000,250.000000,0.000000,0.000000,0.000000,0.000000
75%,2007.000000,26.487500,46.000000,20.000000,43.000000,0.030250,0.046000,11.000000,40.000000,3.467702,6.650000,14.000000,10.000000,0.637500,1.100000,11.500000,10.000000,84.00000,84.000000,67.000000,29.600000,39.873270,-75.226780,83.000000,51.000000,58.000000,646.000000,1859.000000,29.960000,30.100000,21.000000,280.000000,0.000000,0.000000,0.000000,0.000000
max,9997.000000,98.130435,163.000000,23.000000,113.000000,0.079458,0.130000,23.000000,206.000000,53.481250,174.500000,23.000000,176.000000,4.111795,7.850000,23.000000,66.000000,106.00000,119.000000,96.000000,337.400000,40.779200,-73.880000,99.000000,78.000000,80.000000,733.000000,1942.000000,30.730000,30.860000,62.000000,360.000000,4.000000,2.000000,1.000000,3.000000


The categories range from 0 to 4 in this dataset (from Healthy to Very Unhealthy).

#### Ensure the numerical codes are category type.

In [5]:
pollution7['OzoneAQI'] = pollution7['OzoneAQI'].astype('category')
pollution7['NitrogenDioxideAQI'] = pollution7['NitrogenDioxideAQI'].astype('category')
pollution7['CarbonMonoxideAQI'] = pollution7['CarbonMonoxideAQI'].astype('category')
pollution7['SulfurDioxideAQI'] = pollution7['SulfurDioxideAQI'].astype('category')
pollution7.dtypes

/Users/mollymcnamara/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/mollymcnamara/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/mollymcnamara/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

Site_Num                           int64
Date_Local                datetime64[ns]
State                             object
County                            object
City                              object
NO2_Mean                         float64
NO2_1stMaxValue                  float64
NO2_1stMaxHour                   float64
NO2_AQI                          float64
O3_Mean                          float64
O3_1stMaxValue                   float64
O3_1stMaxHour                    float64
O3_AQI                           float64
SO2_Mean                         float64
SO2_1stMaxValue                  float64
SO2_1stMaxHour                   float64
SO2_AQI                          float64
CO_Mean                          float64
CO_1stMaxValue                   float64
CO_1stMaxHour                    float64
CO_AQI                           float64
TempAvg                          float64
TempMax                          float64
TempMin                          float64
Elevation       

### Create train and test sets and define the features of interest

In [6]:
# Identify features of interest
X = pollution7[['TempAvg', 'TempMax', 'TempMin', 'Elevation', 'AvgRelHumid', 'AvgDewPointTemp', 'Sunrise', 'Sunset', 'AvgStationPressure', 'AvgSeaLevelPressure', 'SustainedWindSpeed']]
y_oz = pollution7['OzoneAQI']
y_no = pollution7['NitrogenDioxideAQI']
y_co = pollution7['CarbonMonoxideAQI']
y_so = pollution7['SulfurDioxideAQI']

# Create train and test sets
X_train_oz, X_test_oz, y_train_oz, y_test_oz = train_test_split(X, y_oz, test_size=0.3, random_state=42)
X_train_no, X_test_no, y_train_no, y_test_no = train_test_split(X, y_no, test_size=0.3, random_state=42)
X_train_co, X_test_co, y_train_co, y_test_co = train_test_split(X, y_co, test_size=0.3, random_state=42)
X_train_so, X_test_so, y_train_so, y_test_so = train_test_split(X, y_so, test_size=0.3, random_state=42)

### Modeling

#### Logistic Regression with Imputer

In [7]:
# Setup the pipeline steps: steps
steps = [('imputation', Imputer(missing_values='NaN', strategy='most_frequent', axis=0)),('LR', LogisticRegression())]

# Create the pipeline: pipeline
pipeline = Pipeline(steps)

# Fit the regression to the train set
pipeline.fit(X_train_oz, y_train_oz)
pipeline.fit(X_train_no, y_train_no)
pipeline.fit(X_train_co, y_train_co)
pipeline.fit(X_train_so, y_train_so)

# Predict the labels of the test set
y_pred_oz = pipeline.predict(X_test_oz)
y_pred_no = pipeline.predict(X_test_no)
y_pred_co = pipeline.predict(X_test_co)
y_pred_so = pipeline.predict(X_test_so)

# Compute metrics
print('Ozone Model Accuracy', metrics.accuracy_score(y_test_oz, y_pred_oz))
print('Nitroden Dioxide Model Accuracy', metrics.accuracy_score(y_test_no, y_pred_no))
print('Carbon Monoxide Model Accuracy', metrics.accuracy_score(y_test_co, y_pred_co))
print('Sulfur Dioxide Model Accuracy', metrics.accuracy_score(y_test_so, y_pred_so))

Ozone Model Accuracy 0.913625014861
Nitroden Dioxide Model Accuracy 0.871656164546
Carbon Monoxide Model Accuracy 0.998394958982
Sulfur Dioxide Model Accuracy 0.983355130187


#### Logistic Regression with Cross Validation

In [8]:
# Setup the pipeline
steps = [('imputation', Imputer(missing_values='NaN', strategy='most_frequent', axis=0)),
         ('LR', LogisticRegressionCV())]

pipeline = Pipeline(steps)

# Fit to the training set
pipeline.fit(X_train_oz, y_train_oz)
pipeline.fit(X_train_no, y_train_no)
pipeline.fit(X_train_co, y_train_co)
pipeline.fit(X_train_so, y_train_so)

# Predict the labels of the test set: y_pred
y_pred_oz = pipeline.predict(X_test_oz)
y_pred_no = pipeline.predict(X_test_no)
y_pred_co = pipeline.predict(X_test_co)
y_pred_so = pipeline.predict(X_test_so)

# Compute and print metrics
print('Ozone Model Accuracy: {}'.format(pipeline.score(X_test_oz, y_test_oz)))
print('Nitroden Dioxide Model Accuracy: {}'.format(pipeline.score(X_test_no, y_test_no)))
print('Carbon Monoxide Model Accuracy: {}'.format(pipeline.score(X_test_co, y_test_co)))
print('Sulfur Dioxide Model Accuracy: {}'.format(pipeline.score(X_test_so, y_test_so)))


/Users/mollymcnamara/anaconda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:581: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


Ozone Model Accuracy: 0.913625014861491
Nitroden Dioxide Model Accuracy: 0.8716561645464272
Carbon Monoxide Model Accuracy: 0.9983949589822851
Sulfur Dioxide Model Accuracy: 0.9833551301866603


#### Support Vector Modeling

In [10]:
# Setup the pipeline steps: steps
steps = [('imputation', Imputer(missing_values='NaN', strategy='most_frequent', axis=0)),
        ('SVM', SVC())]

# Create the pipeline: pipeline
pipeline = Pipeline(steps)

# Fit the pipeline to the train set
pipeline.fit(X_train_oz, y_train_oz)
pipeline.fit(X_train_no, y_train_no)
pipeline.fit(X_train_co, y_train_co)
pipeline.fit(X_train_so, y_train_so)

# Predict the labels of the test set
y_pred_oz = pipeline.predict(X_test_oz)
y_pred_no = pipeline.predict(X_test_no)
y_pred_co = pipeline.predict(X_test_co)
y_pred_so = pipeline.predict(X_test_so)

# Compute and print metrics
print('Ozone Model Accuracy', metrics.accuracy_score(y_test_oz, y_pred_oz))
print('Nitroden Dioxide Model Accuracy', metrics.accuracy_score(y_test_no, y_pred_no))
print('Carbon Monoxide Model Accuracy', metrics.accuracy_score(y_test_co, y_pred_co))
print('Sulfur Dioxide Model Accuracy', metrics.accuracy_score(y_test_so, y_pred_so))

Ozone Model Accuracy 0.909582689335
Nitroden Dioxide Model Accuracy 0.870110569492
Carbon Monoxide Model Accuracy 0.994174295565
Sulfur Dioxide Model Accuracy 0.982106764951
